<a href="https://colab.research.google.com/github/manasdeshpande125/da6401_assignment2-partA/blob/main/DL_ASG2_Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Just a simple CNN with 5 layers and Maxpooling built with Pytorch**

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FlexibleCNN(nn.Module):
    def __init__(self,
                 input_shape=(3, 224, 224),
                 num_filters=32,      # m
                 filter_size=3,       # k
                 activation=nn.ReLU,
                 dense_neurons=128,   # n
                 num_classes=10):

        super().__init__()
        self.activation = activation()

        C, H, W = input_shape
        self.conv_blocks = nn.Sequential()
        in_channels = C

        # 5 Conv-Activation-MaxPool blocks
        for i in range(5):
            conv = nn.Conv2d(in_channels, num_filters, kernel_size=filter_size, padding=1)
            self.conv_blocks.add_module(f'conv{i+1}', conv)
            self.conv_blocks.add_module(f'activation{i+1}', activation())
            self.conv_blocks.add_module(f'pool{i+1}', nn.MaxPool2d(kernel_size=2, stride=2))
            in_channels = num_filters

        # Calculate flattened output size
        dummy_input = torch.zeros(1, *input_shape)
        with torch.no_grad():
            out = self.conv_blocks(dummy_input)
        self.flattened_size = out.view(1, -1).shape[1]

        # Dense layers
        self.fc1 = nn.Linear(self.flattened_size, dense_neurons)
        self.output = nn.Linear(dense_neurons, num_classes)

    def forward(self, x):
        x = self.conv_blocks(x)
        x = torch.flatten(x, 1)
        x = self.activation(self.fc1(x))
        x = self.output(x)
        return x


**Assuming Input Size is 224 x 224 x 3
Assuming m=32 i.e number of filters,
k=3 i.e filter_size
and n=128 i.e number of neurons in flattened layer**



In [ ]:
C, H, W = 3, 224, 224
m = 32
k = 3
n = 128
num_classes = 10

def conv_parameters(C_in, C_out, K, num_blocks=5):
    total = 0
    print("Convolutional Layer Parameters:")
    for i in range(num_blocks):
        params = (K * K * C_in + 1) * C_out
        print(f"  Conv Layer {i+1}: {params:,} parameters")
        total += params
        C_in = C_out
    return total

def dense_parameters(flattened_size, dense_neurons, num_classes):
    fc1_params = (flattened_size + 1) * dense_neurons
    out_params = (dense_neurons + 1) * num_classes
    print("\nFully Connected Layer Parameters:")
    print(f"  FC1: {fc1_params:,} parameters")
    print(f"  Output: {out_params:,} parameters")
    return fc1_params + out_params

def conv_computations(H, W, C_in, C_out, K, num_blocks=5):
    total = 0
    print("\nConvolutional Layer Computations:")
    for i in range(num_blocks):
        H, W = H // 2, W // 2  # MaxPool halves the size
        ops = H * W * C_in * K * K * C_out
        print(f"  Conv Layer {i+1}: {ops:,} operations")
        total += ops
        C_in = C_out
    return total, H, W

def dense_computations(flattened_size, dense_neurons, num_classes):
    fc1_ops = flattened_size * dense_neurons
    out_ops = dense_neurons * num_classes
    print("\nFully Connected Layer Computations:")
    print(f"  FC1: {fc1_ops:,} operations")
    print(f"  Output: {out_ops:,} operations")
    return fc1_ops + out_ops

# Run computations
conv_ops, final_H, final_W = conv_computations(224, 224, C, m, k)
flattened = m * final_H * final_W
dense_ops = dense_computations(flattened, n, num_classes)
print()
# Run parameter counts
conv_params = conv_parameters(3, m, k)
dense_params = dense_parameters(flattened, n, num_classes)

print("\nTotal Computations:", f"{conv_ops + dense_ops:,}")
print("Total Parameters:", f"{conv_params + dense_params:,}")



Convolutional Layer Computations:
  Conv Layer 1: 10,838,016 operations
  Conv Layer 2: 28,901,376 operations
  Conv Layer 3: 7,225,344 operations
  Conv Layer 4: 1,806,336 operations
  Conv Layer 5: 451,584 operations

Fully Connected Layer Computations:
  FC1: 200,704 operations
  Output: 1,280 operations

Convolutional Layer Parameters:
  Conv Layer 1: 896 parameters
  Conv Layer 2: 9,248 parameters
  Conv Layer 3: 9,248 parameters
  Conv Layer 4: 9,248 parameters
  Conv Layer 5: 9,248 parameters

Fully Connected Layer Parameters:
  FC1: 200,832 parameters
  Output: 1,290 parameters

Total Computations: 49,424,640
Total Parameters: 240,010


**Calculations as well formula for calculations is specified**



In [ ]:
C, H, W = 3, 224, 224
m = 32
k = 3 #filter_size
n = 128
num_classes = 10

def conv_parameters(C_in, C_out, K, num_blocks=5):
    total = 0
    print("Convolutional Layer Parameters:")
    for i in range(num_blocks):
        params = (K * K * C_in + 1) * C_out
        print(f"  Conv Layer {i+1}:")
        print(f"    Formula: ({K}×{K}×C_in + 1) × {C_out}")
        # print(f"    Where: C_in = {C_in}, C_out = {C_out}, k = {K}")
        print(f"    Params:  {params:,}")
        total += params
        C_in = C_out
    return total

def dense_parameters(flattened_size, dense_neurons, num_classes):
    fc1_params = (flattened_size + 1) * dense_neurons
    out_params = (dense_neurons + 1) * num_classes
    print("\nFully Connected Layer Parameters:")
    print(f"  FC1:")
    print(f"    Formula: (flattened + 1) × n")
    # print(f"    Where: flattened = {flattened_size}, n = {dense_neurons}")
    print(f"    Params:  {fc1_params:,}")
    print(f"  Output Layer:")
    print(f"    Formula: (n + 1) × num_classes")
    # print(f"    Where: n = {dense_neurons}, num_classes = {num_classes}")
    print(f"    Params:  {out_params:,}")
    return fc1_params + out_params

def conv_computations(H, W, C_in, C_out, K, num_blocks=5):
    total = 0
    print("\nConvolutional Layer Computations:")
    for i in range(num_blocks):
        H, W = H // 2, W // 2  # MaxPool halves the size
        ops = H * W * C_in * K * K * C_out
        print(f"  Conv Layer {i+1}:")
        print(f"    Formula: (H × W × C_in × k × k × C_out)")
        # print(f"    Where: H={H}, W={W}, C_in={C_in}, k={K}, C_out={C_out}")
        print(f"    Ops:     {ops:,}")
        total += ops
        C_in = C_out
    return total, H, W

def dense_computations(flattened_size, dense_neurons, num_classes):
    fc1_ops = flattened_size * dense_neurons
    out_ops = dense_neurons * num_classes
    print("\nFully Connected Layer Computations:")
    print(f"  FC1:")
    print(f"    Formula: flattened × n")
    # print(f"    Where: flattened = {flattened_size}, n = {dense_neurons}")
    print(f"    Ops:     {fc1_ops:,}")
    print(f"  Output Layer:")
    print(f"    Formula: n × num_classes")
    # print(f"    Where: n = {dense_neurons}, num_classes = {num_classes}")
    print(f"    Ops:     {out_ops:,}")
    return fc1_ops + out_ops

# Run computations
conv_ops, final_H, final_W = conv_computations(H, W, C, m, k)
flattened = m * final_H * final_W
dense_ops = dense_computations(flattened, n, num_classes)
print()
# Run parameter counts
conv_params = conv_parameters(C, m, k)
dense_params = dense_parameters(flattened, n, num_classes)

print("\nTotal Computations:", f"{conv_ops + dense_ops:,}")
print("Total Parameters:", f"{conv_params + dense_params:,}")



Convolutional Layer Computations:
  Conv Layer 1:
    Formula: (H × W × C_in × k × k × C_out)
    Ops:     10,838,016
  Conv Layer 2:
    Formula: (H × W × C_in × k × k × C_out)
    Ops:     28,901,376
  Conv Layer 3:
    Formula: (H × W × C_in × k × k × C_out)
    Ops:     7,225,344
  Conv Layer 4:
    Formula: (H × W × C_in × k × k × C_out)
    Ops:     1,806,336
  Conv Layer 5:
    Formula: (H × W × C_in × k × k × C_out)
    Ops:     451,584

Fully Connected Layer Computations:
  FC1:
    Formula: flattened × n
    Ops:     200,704
  Output Layer:
    Formula: n × num_classes
    Ops:     1,280

Convolutional Layer Parameters:
  Conv Layer 1:
    Formula: (3×3×C_in + 1) × 32
    Params:  896
  Conv Layer 2:
    Formula: (3×3×C_in + 1) × 32
    Params:  9,248
  Conv Layer 3:
    Formula: (3×3×C_in + 1) × 32
    Params:  9,248
  Conv Layer 4:
    Formula: (3×3×C_in + 1) × 32
    Params:  9,248
  Conv Layer 5:
    Formula: (3×3×C_in + 1) × 32
    Params:  9,248

Fully Connected Layer 